This notebook is used to check the migration of PBs method from SimaPro to Brightway2

For that, its performance will be checked against the dataset of chemicals from Green Chemistry paper.

# Read set of SimaPro solved chemicals

In [2]:
import re
from pathlib import Path

import brightway2 as bw
import pandas as pd
from project_path import ROOT_DIR
import project_path
from src.aesa_pbs.utils import progressbar, solve_lca

In [3]:
import bw2data

In [4]:
from bw_recipe_2016 import add_recipe_2016
import aesa_pbs
aesa_pbs.__version__

'0.8.8+xlsxtoyaml.8.3'

In [5]:
SIMAPRO_DATA = Path(ROOT_DIR + r"\data\raw\lcia-results-from-sp910-combined.xlsx")

In [6]:
with pd.ExcelFile(SIMAPRO_DATA, engine=None) as xl_obj:
    df_simapro = pd.read_excel(xl_obj, sheet_name="Sheet1")

In [7]:
df_simapro.head(3)

,wkbName,Activity,"('PBs-LCIA V0.71 V0.71', 'Climate change - CO2 concentration', 'ppm')","('PBs-LCIA V0.71 V0.71', 'Climate change - Energy imbalance', 'Wm-2')","('PBs-LCIA V0.71 V0.71', 'Stratospheric ozone depletion', 'DU')","('PBs-LCIA V0.71 V0.71', 'Ocean acidification', 'Omega Aragon')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - P', 'Tg P')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - N', 'Tg N')","('PBs-LCIA V0.71 V0.71', 'Land-system change - Global', '%')","('PBs-LCIA V0.71 V0.71', 'Freshwater use - Global', 'km3')",...,"('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Human health', 'DALY')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Ecosystems', 'species.yr')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Resources', 'USD2013')","('IPCC 2013 GWP 100a V1.03', 'IPCC GWP 100a', 'kg CO2 eq')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non renewable, fossil', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, nuclear', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, wind, solar, geothe', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, water', 'MJ')"
0,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RoW}| market for sulfuric acid ...,3.682300e-12,4.965498e-14,1.156094e-15,1.125213e-14,2.047673e-16,3.232508e-14,1.099348e-16,7.588386e-13,...,0.000002,2.389076e-09,0.052662,0.156735,6.316318,0.151773,0.000150,0.121871,0.011526,0.087806
1,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RER}| market for sulfuric acid ...,2.682016e-12,3.621578e-14,8.425343e-16,8.195512e-15,2.162822e-16,3.982650e-14,1.233763e-16,2.679813e-13,...,0.000002,2.382083e-09,0.060525,0.119277,7.175078,0.233407,0.000138,0.073102,0.016586,0.064712
2,Chemicals-Acids (inorganic)-Market,Sulfamic acid {GLO}| market for sulfamic acid ...,5.439020e-11,7.271744e-13,8.913290e-15,1.662025e-13,1.122301e-15,2.008279e-13,5.993186e-16,4.956729e-12,...,0.000007,1.325358e-08,0.272812,2.172702,35.989036,2.025470,0.000812,0.479571,0.171391,0.812977


In [8]:
# df_simapro.columns

In [9]:
[i for i in df_simapro.columns if "ReCiPe 2016 Midpoint" in i]

["('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')",
 "('ReCiPe

In [10]:
[i for i in df_simapro.columns if "0.72" in i]

["('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')",
 "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')",
 "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')",
 "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')",
 "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')",
 "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')",
 "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')"]

```python
# all ReCiPe2016 categories implemented in bw_recipe_2016
# use them to update `map_methods`
[
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Fossil resource scarcity"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Ecosystems"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Marine eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Terrestrial Acidification"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Humans"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land occupation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Freshwater Eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Water consumption"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Hierarchist", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Egalitarian", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land transformation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Egalitarian"),


("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "100 year timescale","Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "1000 year timescale","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Individualist"),
]
```

In [11]:
map_methods = {
    "Activity": "Activity",
    "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'freshwater use', 'global'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'climate change', 'atmospheric CO2 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'change in biosphere integrity', 'functional diversity', 'total'),
    "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'ocean acidification', 'carbonate ion concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'biogeochemical flows', 'phosphorus'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'climate change','energy imbalance at top-of-atmosphere'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'biogeochemical flows', 'nitrogen'),
    "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'ozone depletion', 'stratospheric O3 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')": ('AESA (PBs-LCIA)', str(aesa_pbs.__version__), 'land-system change', 'global'),
    "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')": ('ReCiPe 2016', '1.1 (20180117)', 'Midpoint', 'Global Warming', '100 year timescale', 'Hierarchist'),
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human non-carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Land use', 'm2a crop eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Mineral resource scarcity', 'kg Cu eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fossil resource scarcity', 'kg oil eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Water consumption', 'm3'): "
}

In [12]:
rev_map_methods = {v: k for k, v in map_methods.items()}

In [13]:
df_sp_selected = df_simapro.rename(columns=map_methods)[map_methods.values()]
df_sp_selected.head(3)

C:\Users\vtulus\VirtualEnvs\bw2\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,Activity,"(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, freshwater use, global)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.588386e-13,3.687066e-12,4.280597e-14,1.126669e-14,2.047673e-16,4.971751e-14,3.232508e-14,1.156094e-15,1.099348e-16,0.160737
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.679813e-13,2.685277e-12,3.340084e-14,8.205477e-15,2.162822e-16,3.625857e-14,3.982650e-14,8.425343e-16,1.233763e-16,0.123240
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.956729e-12,5.443214e-11,5.046367e-13,1.663306e-13,1.122301e-15,7.277248e-13,2.008279e-13,8.913290e-15,5.993186e-16,2.202389


In [14]:
len(list(df_sp_selected.Activity))

947

In [15]:
# list(df_simapro.Activity)

In [16]:
pattern = re.compile(r"(^.*) {(.*)}\| (.*) \|")

In [17]:
activities_sp = {}
for i in list(df_sp_selected.Activity):
    mo = re.match(pattern, i)
    if mo:
        ref_prod = mo[1].lower()
        location = mo[2]
        name = mo[3].lower()
        activities_sp[i] = {"reference product": ref_prod, "name": name, "location": location}
# activities_sp

# Prepare bw project

In [18]:
bw.projects

Brightway2 projects manager with 9 objects:
	aachen_fast_calculations
	default
	double_count_electricity
	premise_scenarios_ei35
	premisified_scenarios
	premisified_scenarios_cars
	spreadsheet
	supply chain graph
	test
Use `projects.report()` to get a report on all projects.

In [24]:
# Create a "check_PBsmethod" project or activate it

if not [proj[0] for proj in bw.projects.report() if proj[0]=="check_PBsmethod"]:
    bw.projects.set_current("default")
    bw.projects.copy_project("check_PBsmethod", switch=True)
    
    # delete extra dbs
    del bw.databases['ei35cutoff']
    del bw.databases['ei38apos']
    del bw.databases['ei38cutoff']
else:
    bw.projects.set_current("check_PBsmethod")
print(f"Databases in '{bw.projects.current}' project: {bw.databases.list}")
    
    # Delete "check_PBsmethod" project WHEN FINISHED TESTING!
# bw.projects.set_current("default")
# bw.projects.delete_project(name="check_PBsmethod", delete_dir=True)

# Delete database
# del bw.databases['A_technosphere_flows']

Vacuuming database 
Vacuuming database 
Vacuuming database 
Databases in 'check_PBsmethod' project: ['biosphere3', 'ei35apos']


In [25]:
bw.projects.dir

'C:\\Users\\vtulus\\AppData\\Local\\pylca\\Brightway3\\check_PBsmethod.fbe9669d91325ca0149ce34bf0d8ff48'

In [26]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [27]:
if "ei35apos" in bw.databases:
    print("Database already present! No import is needed.")
else:
    print("Need the apos db, should have been copied from 'default' project.")

Database already present! No import is needed.


In [28]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35apos

In [29]:
force_reinstall = True

if (
    any(
        "AESA" in str(aesa) and str(aesa_pbs.__version__) in str(aesa)
        for aesa in bw.methods
    )
    and not force_reinstall
):
    print("AESA methods already present!")
else:
    aesa_pbs.add_aesa_pbs(verbose=False)

Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'ozone depletion', 'stratospheric O3 concentration')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'biogeochemical flows', 'nitrogen', 'directly fixated')


Writing activities to SQLite3 database:


+------------------------------------------------------------+
| Warning                                                    |
+------------------------------------------------------------+
| The method for quantification of directly fixated nitrogen |
| requires an additional database `A_technosphere_flows`.    |
| This database was not found in the current project.        |
| It will be generated now.                                  |
|                                                            |
| Next steps:                                                |
| 1. Use `aesa_pbs.get_nitrogenous_fertilizers()` to filter  |
|    activities producing nitrogenous fertilizers in a       |
|    specific background database.                           |
| 2. Use `aesa_pbs.update_nitrogen_fertilizer_exchanges()`   |
|    to modify those activities.                             |
|                                                            |
| Optionally,                                          

0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/25/2022 12:37:53
  Finished: 07/25/2022 12:37:53
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.45
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'change in biosphere integrity', 'functional diversity', 'total')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'change in biosphere integrity', 'functional diversity', 'direct land use')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'change in biosphere integrity', 'functional diversity', 'CO2eq emissions')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'land-system change', 'global')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'biogeochemical flows', 'nitrogen', 'inverse modelling, surface water')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'climate change', 'energy imbalance at top-of-atmosphere')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'climate change', 'atmospheric CO2 concentration')
Adding ('AESA (PBs-LCIA)', '0.8.8+xlsxt

In [30]:
if any(rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()):
    print("ReCiPe2016 methods already present!")
else:
    add_recipe_2016()

Adding FreshwaterEcotoxicity
Adding LandOccupation
Adding TerrestrialEcotoxicity
Adding GlobalWarming
Adding StratosphericOzoneDepletion
Adding FreshwaterEutrophication
Adding MarineEutrophication
Adding LandTransformation
Adding MarineEcotoxicity
Adding IonizingRadiation
Adding FossilResourceScarcity
Adding MineralResourceScarcity
Adding TerrestrialAcidification
Adding HumanNoncarcinogenicToxicity
Adding ParticulateMatterFormation
Adding WaterConsumption
Adding HumanCarcinogenicToxicity
Adding OzoneFormationHumans
Adding OzoneFormationEcosystems
Adding single endpoints
Adding aggregated endpoints


In [31]:
recipe2016 = [rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()]
pbs = [aesa for aesa in bw.methods if "AESA" in str(aesa) and str(aesa_pbs.__version__) in str(aesa)]

In [32]:
# for m in recipe2016:
for m in pbs:
    for k, v in rev_map_methods.items():
        if str(k) in str(m):
            print(v)

('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')
('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')
('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')
('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')
('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')
('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')
('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')
('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')


In [33]:
eidb = bw.Database("ei35apos")
# eidb

In [34]:
trial = eidb.random()

In [35]:
trial.as_dict().keys()

dict_keys(['comment', 'classifications', 'activity type', 'activity', 'database', 'filename', 'location', 'name', 'synonyms', 'parameters', 'authors', 'type', 'reference product', 'flow', 'unit', 'production amount', 'code'])

In [36]:
trial["name"]

'treatment of blast furnace gas, in power plant'

In [37]:
trial.as_dict()["name"]

'treatment of blast furnace gas, in power plant'

In [38]:
trial.as_dict()["code"]

'85d5835b8da689ce4e31828a93c5a01d'

In [39]:
trial.key

('ei35apos', '85d5835b8da689ce4e31828a93c5a01d')

In [40]:
# [i for i in trial.biosphere()]

In [41]:
# [i for i in trial.exchanges()]

In [42]:
[i for i in trial.production()]

[Exchange: 1.0 kilowatt hour 'treatment of blast furnace gas, in power plant' (kilowatt hour, ES, None) to 'treatment of blast furnace gas, in power plant' (kilowatt hour, ES, None)>]

In [43]:
# [i for i in trial.substitution()]
# [i for i in trial.technosphere()]

In [44]:
# [i for i in trial.upstream()]

In [45]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S'].keys()

dict_keys(['reference product', 'name', 'location'])

In [46]:
eidb_markets = [activity for activity in eidb if "market" in activity["name"]]

activities_bw = []

for act_sp in progressbar(
    itobj=activities_sp.keys(),
    prefix="Progress: ",
    size=20,
    unit="activity(ies)",
):
    # activities_sp.setdefault("spname", None)
    for act in eidb_markets:
        if (
            activities_sp[act_sp]["reference product"].lower() == str(act["reference product"]).lower()
            and "market" in str(act["name"])
            and activities_sp[act_sp]["location"] == str(act["location"])
        ):
            activities_bw.append(act)
            activities_sp[act_sp].update({"einame": act})


Progress: [🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩] 947/947 activity(ies)
Duration: 0:00:05.512631


In [47]:
[i for i, a in activities_sp.items() if "market" in a.get("einame", "nope")]

[]

In [48]:
# len(
#     set(activities_bw)- set([b["spname"] for b in activities_sp.values() if b.get("spname")])
# )

# activities not returned to activities_sp!!!
# STOPPED HERE!
set(activities_bw)- set([b["einame"] for b in activities_sp.values() if b.get("einame")])

{'market for neutralising agent, sodium hydroxide-equivalent' (kilogram, GLO, None),
 'fraction 8 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'methylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 '1,1-dimethylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 'white spirit to generic market for solvent for paint' (kilogram, GLO, None),
 'praseodymium oxide to generic market for mischmetal' (kilogram, GLO, None),
 'market for vegetable oil, refined' (kilogram, GLO, None),
 'market for solvent for chromatography analysis' (kilogram, GLO, None),
 'market for ethanol, without water, in 99.7% solution state, from fermentation, at service station' (kilogram, CH, None),
 'palm oil, refined, to generic market for vegetable oil' (kilogram, GLO, None),
 'fraction 7 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'heptane to generic market for solvent, orga

In [49]:
len(activities_bw)

978

In [50]:
978-33

945

In [51]:
len(set(activities_bw))

978

In [52]:
871+76

947

In [53]:
len(activities_sp)

947

In [54]:
[i for i in activities_sp if activities_sp[i].get("einame", None)==None]

['Deinking emulsion, in paper production {GLO}| market for | APOS, S',
 'Rare earth concentrate, 70% REO, from bastnasite {GLO}| market for | APOS, U - copy']

In [55]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S']["einame"]

'market for sulfuric acid' (kilogram, RoW, None)

In [56]:
def get_einame(key):
    return activities_sp[key].get("einame", None)#["activity"]

In [57]:
get_einame('Sulfuric acid {RoW}| market for sulfuric acid | APOS, S')

'market for sulfuric acid' (kilogram, RoW, None)

In [58]:
df_sp_selected['einame'] = df_sp_selected['Activity'].apply(lambda key: get_einame(key))
df_sp_selected

,Activity,"(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, freshwater use, global)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)",einame
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.588386e-13,3.687066e-12,4.280597e-14,1.126669e-14,2.047673e-16,4.971751e-14,3.232508e-14,1.156094e-15,1.099348e-16,0.160737,"[comment, classifications, activity type, acti..."
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.679813e-13,2.685277e-12,3.340084e-14,8.205477e-15,2.162822e-16,3.625857e-14,3.982650e-14,8.425343e-16,1.233763e-16,0.123240,"[comment, classifications, activity type, acti..."
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.956729e-12,5.443214e-11,5.046367e-13,1.663306e-13,1.122301e-15,7.277248e-13,2.008279e-13,8.913290e-15,5.993186e-16,2.202389,"[comment, classifications, activity type, acti..."
3,"Phosphoric acid, industrial grade, without wat...",2.076097e-11,3.683377e-11,7.170159e-13,1.125543e-13,5.740347e-14,4.924900e-13,4.259820e-13,6.920908e-15,8.400603e-16,1.491608,"[comment, classifications, activity type, acti..."
4,"Phosphoric acid, fertiliser grade, without wat...",1.415177e-11,2.322850e-11,5.772262e-13,7.098025e-14,5.402468e-14,3.118323e-13,3.214601e-13,4.997489e-15,8.019743e-16,0.957087,"[comment, classifications, activity type, acti..."
...,...,...,...,...,...,...,...,...,...,...,...,...
942,"Polyvinylchloride, suspension polymerised {GLO...",4.007584e-12,5.098499e-11,4.271375e-13,1.557991e-13,4.873162e-15,6.738817e-13,1.941198e-13,7.519967e-15,3.080851e-17,2.035933,"[comment, classifications, activity type, acti..."
943,"Polyvinylidenchloride, granulate {RER}| market...",1.261049e-12,1.167023e-10,1.113848e-12,3.566084e-13,6.547821e-15,1.545165e-12,1.637795e-13,1.837176e-15,-2.863068e-18,5.330478,"[comment, classifications, activity type, acti..."
944,"Polyvinylidenchloride, granulate {RoW}| market...",1.275448e-12,1.183721e-10,1.133030e-12,3.617105e-13,6.524817e-15,1.566875e-12,1.181972e-13,1.901179e-15,1.994662e-17,5.399748,"[comment, classifications, activity type, acti..."
945,"Tetrafluoroethylene {GLO}| market for | APOS, U",2.032338e-11,1.900741e-10,2.871105e-11,5.808169e-13,9.836993e-15,6.185484e-12,2.573934e-12,3.205619e-11,1.585247e-15,137.134320,"[comment, classifications, activity type, acti..."


In [59]:
df_sp_selected = df_sp_selected.iloc[[i for i in df_sp_selected.index if df_sp_selected.einame[i] is not None]]

In [60]:
# type(df_sp_selected.einame[665])

In [61]:
# type(activities_bw[0])

In [62]:
df_sp_selected.shape

(945, 12)

In [63]:
c = 0
for idx in df_sp_selected.index:
    if not isinstance(df_sp_selected.einame[idx], bw2data.backends.peewee.proxies.Activity):
        print(df_sp_selected.Activity[idx])
        print(idx)
        c += 1
c

0

In [64]:
dct_FUs = [{act: 1} for act in df_sp_selected.einame] # if isinstance(act, bw2data.backends.peewee.proxies.Activity)]
# dct_FUs = [{act: 1} for act in activities_bw]
# dct_FUs

In [65]:
# FU0 = dct_FUs[45]
# all_scores = {}
# lca = bw.LCA(FU0, pbs[0])
# lca.lci()
# lca.lcia()
# for category in pbs:
#     lca.switch_method(category)
#     lca.lcia()
#     all_scores[category] = {}
#     all_scores[category]["score"] = lca.score
#     all_scores[category]["unit"] = bw.Method(category).metadata["unit"]

In [66]:
# FU0

In [67]:
pd.options.display.float_format = '{:12.5e}'.format

In [68]:
# df_temp = pd.DataFrame.from_dict(all_scores).T
# df_temp

In [69]:
bw.calculation_setups

CalculationSetups dictionary with 0 objects

In [70]:
pbs + ["'ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq'"]

[('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'ozone depletion',
  'stratospheric O3 concentration'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'biogeochemical flows',
  'nitrogen',
  'directly fixated'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'total'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'direct land use'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'CO2eq emissions'),
 ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'land-system change', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'biogeochemical flows',
  'nitrogen',
  'inverse modelling, surface water'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'climate change',
  'energy imbalance at top-of-atmosphere'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'climate change',
  'atmospheric CO2 conc

In [71]:
selected_methods = pbs + [
    m for k, v in rev_map_methods.items() for m in recipe2016 if str(k) in str(m)
]

In [72]:
bw.calculation_setups["chemicals"] = {
    "inv": dct_FUs,
    "ia": selected_methods,
}

In [73]:
# [act["name"] for FU in myMultiLCA.func_units for act in FU.keys()]

In [74]:
# bw.calculation_setups["chemicals"]
myMultiLCA = bw.MultiLCA("chemicals")
# myMultiLCA.results

df_impact = pd.DataFrame(
    data=myMultiLCA.results,
    columns=selected_methods,
    # index=[[act for FU in myMultiLCA.func_units for act in FU.keys()]]
    index=[
        tuple([act["name"], act["location"], act["reference product"]])
        for FU in myMultiLCA.func_units
        for act in FU
    ],
)
df_impact.head(3)

,"(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, nitrogen, directly fixated)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, change in biosphere integrity, functional diversity, total)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, change in biosphere integrity, functional diversity, direct land use)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, change in biosphere integrity, functional diversity, CO2eq emissions)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, land-system change, global)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, nitrogen, inverse modelling, surface water)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, freshwater use, global)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), 0.8.8+xlsxtoyaml.8.3, biogeochemical flows, phosphorus)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
"(market for sulfuric acid, RoW, sulfuric acid)",1.15609e-15,0.00000e+00,4.59113e-14,1.25277e-14,3.33836e-14,1.09935e-16,4.67079e-14,4.97176e-14,3.68707e-12,2.02947e-11,1.12667e-14,2.04767e-16,1.60736e-01
"(market for sulfuric acid, RER, sulfuric acid)",8.42535e-16,0.00000e+00,3.62595e-14,1.06638e-14,2.55958e-14,1.23376e-16,4.97176e-14,3.62586e-14,2.68528e-12,2.59210e-11,8.20548e-15,2.16282e-16,1.23239e-01
"(market for sulfamic acid, GLO, sulfamic acid)",8.91329e-15,0.00000e+00,5.21511e-13,6.40927e-14,4.57418e-13,5.99318e-16,1.29056e-12,7.27725e-13,5.44322e-11,9.56326e-11,1.66331e-13,1.12230e-15,2.20238e+00


In [75]:
df_impact.shape

(945, 13)

In [76]:
# [(act["name"], act["location"]) for FU in myMultiLCA.func_units for act in FU]

In [77]:
indices = [tuple([act["name"], act["location"], act["reference product"]]) for act in df_sp_selected.einame]

In [78]:
# df_sp_selected

In [79]:
df_sp_selected["einame_idx"] = indices

C:\Users\vtulus\AppData\Local\Temp\ipykernel_12028\218076019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected["einame_idx"] = indices


In [80]:
df_sp_selected.set_index("einame_idx", inplace=True)
df_sp_selected.drop("Activity", 1, inplace=True)
df_sp_selected.drop("einame", 1, inplace=True)

C:\Users\vtulus\AppData\Local\Temp\ipykernel_12028\3607502913.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\vtulus\AppData\Local\Temp\ipykernel_12028\3607502913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\vtulus\AppData\Local\Temp\ipykernel_12028\3607502913.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("einame", 1, inplace=True)
C:\Users\vtulus\AppData\Local\Temp\ipykernel_12028\3607502913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [81]:
df_sp_selected.shape

(945, 10)

In [82]:
import numpy as np

In [83]:
# np.isclose(7.40056E-13*10e13,1.05827e-13*10e13)

In [84]:
df_impact.shape

(945, 13)

# one-by-one comparisons

In [85]:
def reindex_pbs_scores(data: pd.DataFrame) -> pd.DataFrame:
    to_return = data.reindex(
        columns=[
            "climate change - atmospheric CO2 concentration",
            "climate change - energy imbalance at top-of-atmosphere",
            "ozone depletion - stratospheric O3 concentration",
            "ocean acidification - carbonate ion concentration",
            "biogeochemical flows - phosphorus",
            "biogeochemical flows - nitrogen",
            "land-system change - global",
            "freshwater use - global",
            "change in biosphere integrity - functional diversity",
        ],
        level=0,
    ).T
    return to_return

In [86]:
# axt = [
#     a
#     for a in eidb
#     if "heat and power co-generation, wood chips, 6667 kW, state" in a["name"]
#     and "electricity" in a["reference product"]
#     and "ES" == a["location"]
#     and "label-certified" not in a["name"]
# ]  # , 'green manure, Swiss integrated production, until March')

# axt = [a for a in eidb if "market for sulfuric acid" in a["name"] and "RER" in a["location"]]

# axt = [a for a in eidb if "market for silicon, multi-Si, casted" in a["name"] and "CA-QC" in a["location"]]

# axt = [a for a in eidb if "lime to generic market for soil pH raising agent" in a["name"] and "GLO" in a["location"]]
axt = [a for a in eidb if "market for potassium fertiliser, as K2O" in a["name"] and "GLO" in a["location"]]

# axt = [
#     a
#     for a in eidb
#     if "market for green manure, Swiss integrated production, until March" in a["name"]
#     and "GLO" in a["location"]
# ]

axt

['market for potassium fertiliser, as K2O' (kilogram, GLO, None)]

In [87]:
results_df = reindex_pbs_scores(solve_lca(axt, pbs))
results_df

,Name,"market for potassium fertiliser, as K2O"
,Location,GLO
Method,Unit,
climate change - atmospheric CO2 concentration,ppm,3.86726e-11
climate change - energy imbalance at top-of-atmosphere,Wm-2,5.83656e-13
ozone depletion - stratospheric O3 concentration,Dobson unit,2.19451e-13
ocean acidification - carbonate ion concentration,omega aragonite,1.18167e-13
biogeochemical flows - phosphorus,Tg P,1.04999e-13
biogeochemical flows - nitrogen,Tg N,1.50788e-11
land-system change - global,% forested land,1.27769e-14
freshwater use - global,km3,1.37133e-10


In [88]:
sp_results = {
    "climate change - atmospheric CO2 concentration": 3.8672584E-11,
    "climate change - energy imbalance at top-of-atmosphere": 5.8365615E-13,
    "ozone depletion - stratospheric O3 concentration":2.1945121E-13,
    "ocean acidification - carbonate ion concentration": 1.1816742E-13,
    "biogeochemical flows - phosphorus": 1.0499918E-13,
    "biogeochemical flows - nitrogen": 3.0157583E-11,
    "land-system change - global": 1.2776855E-14,
    "freshwater use - global": 1.370915E-10,
    "change in biosphere integrity - functional diversity": 1.6649877E-12,
}
sp_results_df = pd.DataFrame().from_dict(
    sp_results,
    orient="index",
    columns=[results_df.columns[0][0]],
)
sp_results_df

,"market for potassium fertiliser, as K2O"
climate change - atmospheric CO2 concentration,3.86726e-11
climate change - energy imbalance at top-of-atmosphere,5.83656e-13
ozone depletion - stratospheric O3 concentration,2.19451e-13
ocean acidification - carbonate ion concentration,1.18167e-13
biogeochemical flows - phosphorus,1.04999e-13
biogeochemical flows - nitrogen,3.01576e-11
land-system change - global,1.27769e-14
freshwater use - global,1.37092e-10
change in biosphere integrity - functional diversity,1.66499e-12


In [89]:
# relative difference of bw vs sp results
# negative - in bw is less than in sp
# positive - in bw is more than in sp

bw_res = results_df.droplevel(1, axis=0).droplevel(1, axis=1)
sp_res = sp_results_df
rel_diff = (bw_res-sp_res)/sp_res*100
rel_diff.iloc[:,0] = rel_diff.iloc[:,0].map('{:.2f}%'.format)
rel_diff

Name,"market for potassium fertiliser, as K2O"
Method,
climate change - atmospheric CO2 concentration,-0.00%
climate change - energy imbalance at top-of-atmosphere,-0.00%
ozone depletion - stratospheric O3 concentration,0.00%
ocean acidification - carbonate ion concentration,-0.00%
biogeochemical flows - phosphorus,0.00%
biogeochemical flows - nitrogen,-50.00%
land-system change - global,-0.00%
freshwater use - global,0.03%
change in biosphere integrity - functional diversity,-30.75%


In [90]:
from collections import defaultdict


def top_emissions_by_name(lca, biosphere_database="biosphere3"):
    names = defaultdict(list)

    for flow in bw.Database(biosphere_database):
        if flow.key in lca.biosphere_dict:
            names[flow["name"]].append(
                lca.characterized_inventory[lca.biosphere_dict[flow.key], :].sum()
            )

    return sorted([(sum(scores), name) for name, scores in names.items()], reverse=True)

In [91]:
pbs

[('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'ozone depletion',
  'stratospheric O3 concentration'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'biogeochemical flows',
  'nitrogen',
  'directly fixated'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'total'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'direct land use'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'change in biosphere integrity',
  'functional diversity',
  'CO2eq emissions'),
 ('AESA (PBs-LCIA)', '0.8.8+xlsxtoyaml.8.3', 'land-system change', 'global'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'biogeochemical flows',
  'nitrogen',
  'inverse modelling, surface water'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'climate change',
  'energy imbalance at top-of-atmosphere'),
 ('AESA (PBs-LCIA)',
  '0.8.8+xlsxtoyaml.8.3',
  'climate change',
  'atmospheric CO2 conc

In [92]:
lca = bw.LCA({axt[0]: 1}, pbs[1])
lca.lci()
lca.lcia()
lca.score

0.0

In [93]:
lca.demand

{'market for potassium fertiliser, as K2O' (kilogram, GLO, None): 1}

In [94]:
(7.010726210309949e-14-4.38e-14)/4.38e-14*(7.010726210309949e-14/lca.score)

ZeroDivisionError: float division by zero

In [95]:
5.522868070172049e-16/lca.score

ZeroDivisionError: float division by zero

In [96]:
top_emissions_by_name(lca)[:50]

[(0.0, 'tau-Fluvalinate'),
 (0.0, 't-Butylamine'),
 (0.0, 't-Butyl methyl ether'),
 (0.0, 'o-Xylene'),
 (0.0, 'o-Nitrotoluene'),
 (0.0, 'o-Dichlorobenzene'),
 (0.0, 'm-Xylene'),
 (0.0, '[Deleted]Tri-allate'),
 (0.0, '[Deleted]Carfentrazone ethyl ester'),
 (0.0, 'Zirconium-95'),
 (0.0, 'Zirconium, 50% in zircon, 0.39% in crude ore, in ground'),
 (0.0, 'Zirconium'),
 (0.0, 'Zinc-65'),
 (0.0, 'Zinc, ion'),
 (0.0, 'Zinc, Zn 3.1%, in mixed ore, in ground'),
 (0.0, 'Zinc, Zn 0.63%, in mixed ore, in ground'),
 (0.0, 'Zinc, 9.0% in sulfide, Zn 5.3%, Pb, Ag, Cd, In, in ground'),
 (0.0, 'Zinc'),
 (0.0, 'Zeta-cypermethrin'),
 (0.0, 'Xylene'),
 (0.0, 'Xenon-138'),
 (0.0, 'Xenon-137'),
 (0.0, 'Xenon-135m'),
 (0.0, 'Xenon-135'),
 (0.0, 'Xenon-133m'),
 (0.0, 'Xenon-133'),
 (0.0, 'Xenon-131m'),
 (0.0, 'Xenon, in air'),
 (0.0, 'Wood, unspecified, standing'),
 (0.0, 'Wood, soft, standing'),
 (0.0, 'Wood, hard, standing'),
 (0.0, 'Water, well, in ground'),
 (0.0, 'Water, unspecified natural origin'),
 (0

In [97]:
# contributions in sp can be found in the "Inventory" tab

# for Electricity, high voltage {CH}| heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014 | APOS, U
# that yields the same score in CBI
# the main contributor is "Occupation, forest, extensive" ~94% of impact

# on the other hand, for 'market for sulfuric acid' (kilogram, RoW, None) the ~65% of impact is 
# of "Carbon dioxide, fossil" in sp, BUT NOT IN bw!!!
# so the problem is the ReCiPe method???
# in bw "Carbon dioxide, fossil" yields 8.2e-15
# while in sp - 2.78e-14

In [98]:
l = [t[0] for t in top_emissions_by_name(lca)[1:]]
print(sum(l))
# np.isclose(sum(l), lca.score)
1-sum(l)/lca.score

0.0


nan

In [99]:
(8.281146767077711e-15+8.237074606492378e-15+1.8354772572644123e-15+1.3654838175278122e-15)/lca.score

ZeroDivisionError: float division by zero

In [100]:
2e-14 == 0.2e-13

True

In [101]:
import bw2analyzer as bwa

In [102]:
# see impact contributors to an activity group by CPC classification
# if "relative" - shows fractional contribution to total

# bwa.compare_activities_by_grouped_leaves(axt, pbs[2], mode="relative", output_format="list", max_level=5)

In [103]:
ca = bwa.ContributionAnalysis()
# ca.annotated_top_processes(lca, limit=5)
ca.annotated_top_emissions(lca, limit=0.00001, limit_type='percent')

[(0.0,
  2.062516971172559e-09,
  '[Deleted]Carfentrazone ethyl ester' (kilogram, None, ('soil', 'agricultural'))),
 (0.0,
  2.6653032000054992e-06,
  'Zinc, ion' (kilogram, None, ('water', 'ground-'))),
 (0.0,
  3.8115643913006945e-08,
  'Trifloxystrobin' (kilogram, None, ('soil', 'agricultural'))),
 (0.0,
  1.040549903568664e-06,
  'Trifluralin' (kilogram, None, ('soil', 'agricultural'))),
 (0.0,
  9.002592851502329e-10,
  'Trimethylamine' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.0,
  4.243628166227408e-07,
  'Trinexapac-ethyl' (kilogram, None, ('soil', 'agricultural'))),
 (0.0,
  4.0241856990623803e-07,
  'Ulexite, in ground' (kilogram, None, ('natural resource', 'in ground'))),
 (0.0,
  4.440341198790047e-09,
  'Uranium' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.0,
  2.313463017484098e-05,
  'Uranium-238' (kilo Becquerel, None, ('air', 'urban air close to ground'))),
 (0.0,
  1.6139852712083686e-06,
  'Vanadium' (kilogram, None, ('air', 'urban

In [104]:
-4.373743127925108e-11+1.1730336203662644e-11+1.1320238491218266e-11+8.922289063820765e-12+6.830495504523874e-12

-4.934072016025528e-12

# results

In [105]:
res = (df_impact-df_sp_selected).div(df_sp_selected).mul(100)

In [106]:
# res

In [107]:
def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=-100, vmax=100, cmap="seismic")
    return styler

In [108]:
# big diffrences in: 
# "market for green manure" due to diffenent units, bw -> ha, simapro -> m2
# lime to generic market for soil pH raising agent: recalculated:OK
# methyl acetate to generic market for solvent, organic', 'GLO: recalculated: OK

In [109]:
res.style.pipe(make_pretty)